# Price ML Predicion

## Price Prediction's Caveats

  When the characteristics of the instrument are far from stable, price prediction can be quite off.
  - Examples...
    - SERV
    - MEDS
    - XCUR
  
  Price Prediction should be combined with Prediction of market highs and lows because if you just entered the market based on predicted price, you could get washed out by manipulative Market Wales hitting your stops.
  
  Predicting High, Low, Market Open and Market Close Should be done using separate models, using the same model parameters.
  
  Before trading an instrument, one should review the median market price range, verses median prediction accuracy delta, to verify whether the instrument is worth trading. I have a hunch that the smaller the trading range as compared to the prediction delta, the less likely the instrument is worth engaging with.



# Choose an existing model -or- Create a New Model


In [1]:
# Create a list of available .keras models from model_dir...
# Restart the kernel
%reset -f
%gui asyncio

import ipywidgets as widgets
from ipywidgets import Output  
import re
import datetime
import os
import asyncio
import yfinance as yf

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future


Ticker    = '271060.KS' 
Ticker    = 'GIB'
Ticker    = 'TSLA'
Ticker    = 'AAPL'
Ticker    = 'MGC=F'

DateStart = '2015-06-03'
DateEnd   = '2024-07-30'   
CreateNewModel = "Create New Model"

# Get list of file from the model directory...
model_dir = 'models/'
model_files = os.listdir(model_dir)
# Filter the list to only include .keras files...
model_files = [CreateNewModel] + [file for file in model_files if re.search(r'\.keras$', file)]
# Display the list of files...
Selected_Model = widgets.Select ( options=model_files,
                                  value=model_files[0],
                                  description='Model File: ',
                                  disabled=False )
Ticker     = widgets.Text(value=Ticker, description='TICKER: ')
DateStart = "2015-01-01"
# Set the DateEnd to today's date with the format 'YYYY-MM-DD, determined by the system date.
DateEnd   = datetime.datetime.now().strftime('%Y-%m-%d')
DateStart = widgets.Text(value=DateStart, description='Start Date: ')
DateEnd   = widgets.Text(value=DateEnd, description='End Date: ')
out = Output()

async def f0():
    for i in range(10):
        out.append_stdout('did work ' + str(i) + '\n')
        x = await wait_for_change(Selected_Model, 'value')
        if Selected_Model.value != CreateNewModel:
            Ticker.value = Selected_Model.value.split('_')[0]
        out.append_stdout('async function continued with value ' + str(x) + '\n')
    asyncio.ensure_future(f0())

async def f1():
    for i in range(10):
        out.append_stdout('did work ' + str(i) + '\n')
        x = await wait_for_change(Ticker, 'value')    
        tkr = Ticker.value
        tkr = re.sub(r'Invalid: ', '', tkr)
        if Selected_Model.value == CreateNewModel:
            # Check yahoo if symbol is valid...
            try:
                ticker = yf.Ticker(tkr).history(period='5d',interval='1d')
            except:
                ticker = []
            if len(ticker) == 0:
                Ticker.value = 'Invalid: ' + tkr
            else:
                Ticker.value = tkrxxxx.upper()
        out.append_stdout('async function continued with value ' + str(x) + '\n')
    asyncio.ensure_future(f1())

asyncio.ensure_future(f0())
asyncio.ensure_future(f1())

inst0 = """   
    +-------------------------------+" +
    | Choose or Create a New Model  |
    +-------------------------------+
"""
inst1 = """Make you Model Selection and Updates the Start and End Dates.
Or, Create a New Model by entering by leave the Model File: as 
"Create New Model", and entering the Ticker, Start Date and End Date.
"""
print(inst0, inst1) 
display(Selected_Model)
print() 
display(Ticker, DateStart, DateEnd)
print()
display(out)            

# if selected_model.value == CreateNewModel:
# - Create a new model using the Ticker_vale.value, DateStart.value, DateEnd.vale values.


FileNotFoundError: [Errno 2] No such file or directory: 'models/'

In [2]:
print( "Selected_Model:", Selected_Model.value)
print("Ticker: ", Ticker.value)
print("DateStart: ", DateStart.value)
print("DateEnd: ", DateEnd.value)


Selected_Model: Create New Model
Ticker:  MGC=F
DateStart:  2015-01-01
DateEnd:  2024-07-26


#  Load the latest data for the selected Ticker...

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import sklearn

model_dir = 'models/'

if Selected_Model.value == CreateNewModel:
    modelDateStart = DateStart
    modelDateEnd = DateEnd
    model_file = Ticker.value + "_" + DateStart.value + "_" + DateEnd.value + ".keras"
    model_path = model_dir + model_file
else:
    model_path = Selected_Model.value

# If we are using an existing model, we still have to download the data as we need the
# actual price data to adjust the prediction.

# If the predicted value varies/swings-about too much from the price,
# the days/period training period many be changed to exclude the prior period
# That does not have present-time characteristics with regard to price movement.
data = yf.download(tickers = Ticker.value, start = DateStart.value, end = DateEnd.value)
offset = pd.Timedelta(days=-30)
# Resample to 'W'eekly or 'ME'(Month End)
# logic = {'Open'  : 'first',
#          'High'  : 'max',
#          'Low'   : 'min',
#          'Close' : 'last',
#          'Adj Close': 'last',
#          'Volume': 'sum'}
# data = data.resample('W', offset=offset).apply(logic)
print(len(data))

data.tail(10)
    

[*********************100%%**********************]  1 of 1 completed

2405


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-12,2420.800049,2423.199951,2396.199951,2420.699951,2420.699951,84634
2024-07-15,2419.300049,2445.000000,2406.100098,2428.899902,2428.899902,96193
2024-07-16,2427.199951,2474.600098,2424.500000,2467.800049,2467.800049,107637
2024-07-17,2473.000000,2488.399902,2455.600098,2459.899902,2459.899902,134906
2024-07-18,2462.000000,2478.500000,2442.899902,2456.399902,2456.399902,97615
2024-07-19,2447.199951,2448.199951,2395.699951,2399.100098,2399.100098,125294
2024-07-22,2403.699951,2414.399902,2385.100098,2394.699951,2394.699951,91725
2024-07-23,2398.000000,2413.399902,2389.100098,2407.300049,2407.300049,82537
2024-07-24,2411.300049,2433.000000,2396.899902,2415.699951,2415.699951,103221


##  - Add indicators to the data


In [4]:
# Add one row at the end.
# A copy of the last row.
pass

# Adding indicators
IndCount = 0
data['RSI']=ta.rsi(data.Close, length=3); IndCount += 1
# data['EMAF']=ta.ema(data.Close, length=3); IndCount += 1
# data['EMAM']=ta.ema(data.Close, length=6); IndCount += 1
# data['EMAS']=ta.ema(data.Close, length=9); IndCount += 1
data['DPO']=ta.dpo(data.Close, length=3, centered=True); IndCount += 1
data['DPO']=ta.dpo(data.Close, length=6, centered=True); IndCount += 1
data['DPO']=ta.dpo(data.Close, length=9, centered=True); IndCount += 1
if data['Volume'].iloc[-1] > 0:
    data = data.join(ta.aobv(data.Close, data.Volume, fast=True, min_lookback=3, max_lookback=9))
    IndCount += 7 # ta.aobv adds 7 columns

data['Target'] = data['Adj Close']-data.Open
data['Target'] = data['Target'].shift(-1)

# data.head(10)

data['TargetClass'] = [1 if data['Target'][i]>0 else 0 for i in range(len(data))]

data['TargetNextClose'] = data['Adj Close'].shift(-1)
# 
data.dropna(inplace=True)
data.reset_index(inplace = True)
# data.drop(['Volume', 'Close', 'Date'], axis=1, inplace=True)
data_date = data['Date'].copy()
data.drop(['Date'], axis=1, inplace=True)
# Add one more row to the data file, this will be our next day's prediction.
data = pd.concat([data,data[-1:]])
# And, reindex the dataframe.
data.reset_index(inplace=True)

data.tail(10)

/tmp/ipykernel_369178/314291538.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data['TargetClass'] = [1 if data['Target'][i]>0 else 0 for i in range(len(data))]


,index,Open,High,Low,Close,Adj Close,Volume,RSI,DPO,OBV,OBV_min_3,OBV_max_9,OBVe_1,OBVe_12,AOBV_LR_2,AOBV_SR_2,Target,TargetClass,TargetNextClose
2380,2380,2396.500000,2399.199951,2358.300049,2363.500000,2363.500000,81846,50.707628,-23.511041,2628393.0,2587779.0,2710239.0,2628393.0,2.601590e+06,1,0,1.299805,1,2367.899902
2381,2381,2366.600098,2378.600098,2355.899902,2367.899902,2367.899902,77826,54.560101,-34.044488,2706219.0,2628393.0,2710239.0,2706219.0,2.617687e+06,0,1,8.500000,1,2379.699951
2382,2382,2371.199951,2393.300049,2369.699951,2379.699951,2379.699951,63247,65.429348,-32.299995,2769466.0,2628393.0,2769466.0,2769466.0,2.641037e+06,1,0,44.799805,1,2421.899902
2383,2383,2377.100098,2430.399902,2376.800049,2421.899902,2421.899902,108816,84.858432,3.377740,2878282.0,2706219.0,2878282.0,2878282.0,2.677537e+06,1,0,-0.100098,0,2420.699951
2384,2384,2420.800049,2423.199951,2396.199951,2420.699951,2420.699951,84634,82.871903,-1.777778,2793648.0,2769466.0,2878282.0,2793648.0,2.695400e+06,1,0,9.599854,1,2428.899902
2385,2385,2419.300049,2445.000000,2406.100098,2428.899902,2428.899902,96193,86.186573,3.444390,2889841.0,2793648.0,2889841.0,2889841.0,2.725314e+06,1,0,40.600098,1,2467.800049
2386,2386,2427.199951,2474.600098,2424.500000,2467.800049,2467.800049,107637,94.188935,39.277859,2997478.0,2793648.0,2997478.0,2997478.0,2.767185e+06,1,0,-13.100098,0,2459.899902
2387,2387,2473.000000,2488.399902,2455.600098,2459.899902,2459.899902,134906,80.060053,32.066596,2862572.0,2862572.0,2997478.0,2862572.0,2.781860e+06,0,1,-5.600098,0,2456.399902
2388,2388,2462.000000,2478.500000,2442.899902,2456.399902,2456.399902,97615,72.802680,36.033257,2764957.0,2764957.0,2997478.0,2764957.0,2.779260e+06,0,1,-48.099854,0,2399.100098
2389,2388,2462.000000,2478.500000,2442.899902,2456.399902,2456.399902,97615,72.802680,36.033257,2764957.0,2764957.0,2997478.0,2764957.0,2.779260e+06,0,1,-48.099854,0,2399.100098


In [5]:
data_set = data.iloc[:, 0:10 + IndCount]#.values
# pd.set_option('display.max_columns', None)

data_set.tail(10)
# print(data_set.shape)
# print(data.shape)
# print(type(data_set))

,index,Open,High,Low,Close,Adj Close,Volume,RSI,DPO,OBV,OBV_min_3,OBV_max_9,OBVe_1,OBVe_12,AOBV_LR_2,AOBV_SR_2,Target,TargetClass,TargetNextClose
2380,2380,2396.500000,2399.199951,2358.300049,2363.500000,2363.500000,81846,50.707628,-23.511041,2628393.0,2587779.0,2710239.0,2628393.0,2.601590e+06,1,0,1.299805,1,2367.899902
2381,2381,2366.600098,2378.600098,2355.899902,2367.899902,2367.899902,77826,54.560101,-34.044488,2706219.0,2628393.0,2710239.0,2706219.0,2.617687e+06,0,1,8.500000,1,2379.699951
2382,2382,2371.199951,2393.300049,2369.699951,2379.699951,2379.699951,63247,65.429348,-32.299995,2769466.0,2628393.0,2769466.0,2769466.0,2.641037e+06,1,0,44.799805,1,2421.899902
2383,2383,2377.100098,2430.399902,2376.800049,2421.899902,2421.899902,108816,84.858432,3.377740,2878282.0,2706219.0,2878282.0,2878282.0,2.677537e+06,1,0,-0.100098,0,2420.699951
2384,2384,2420.800049,2423.199951,2396.199951,2420.699951,2420.699951,84634,82.871903,-1.777778,2793648.0,2769466.0,2878282.0,2793648.0,2.695400e+06,1,0,9.599854,1,2428.899902
2385,2385,2419.300049,2445.000000,2406.100098,2428.899902,2428.899902,96193,86.186573,3.444390,2889841.0,2793648.0,2889841.0,2889841.0,2.725314e+06,1,0,40.600098,1,2467.800049
2386,2386,2427.199951,2474.600098,2424.500000,2467.800049,2467.800049,107637,94.188935,39.277859,2997478.0,2793648.0,2997478.0,2997478.0,2.767185e+06,1,0,-13.100098,0,2459.899902
2387,2387,2473.000000,2488.399902,2455.600098,2459.899902,2459.899902,134906,80.060053,32.066596,2862572.0,2862572.0,2997478.0,2862572.0,2.781860e+06,0,1,-5.600098,0,2456.399902
2388,2388,2462.000000,2478.500000,2442.899902,2456.399902,2456.399902,97615,72.802680,36.033257,2764957.0,2764957.0,2997478.0,2764957.0,2.779260e+06,0,1,-48.099854,0,2399.100098
2389,2388,2462.000000,2478.500000,2442.899902,2456.399902,2456.399902,97615,72.802680,36.033257,2764957.0,2764957.0,2997478.0,2764957.0,2.779260e+06,0,1,-48.099854,0,2399.100098


## - Scale the data

  The data is scaled to a range of 0 to 1.
  This is done to ensure that the data is normalized, so that the model can be trained on it

In [6]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data_set)
print("data_set.shape:", data_set.shape, "data_set_scaled.shape", data_set_scaled.shape)
print(data_set_scaled)

data_set.shape: (2390, 19) data_set_scaled.shape (2390, 19)
[[0.00000000e+00 1.63884959e-01 1.64926093e-01 ... 4.98727114e-01
  0.00000000e+00 1.71418421e-01]
 [4.18760469e-04 1.73842934e-01 1.71164282e-01 ... 5.47581952e-01
  1.00000000e+00 1.76358435e-01]
 [8.37520938e-04 1.72019670e-01 1.72566119e-01 ... 4.56488301e-01
  0.00000000e+00 1.70642150e-01]
 ...
 [9.99581240e-01 1.00000000e+00 1.00000000e+00 ... 4.76335132e-01
  0.00000000e+00 9.91954731e-01]
 [1.00000000e+00 9.92286115e-01 9.93060978e-01 ... 2.60051388e-01
  0.00000000e+00 9.51517325e-01]
 [1.00000000e+00 9.92286115e-01 9.93060978e-01 ... 2.60051388e-01
  0.00000000e+00 9.51517325e-01]]


## - Prepare the scaled data for the model

  - The data is prepared for the model by creating a 3D array of the data.
  - The data is split into training and test data.
  - The training data is used to train the model.
  - The test data is used to test the model.
  - The model is then used to predict the next day's price.

In [7]:
# multiple feature from data provided to the model
X = []
# print(data_set_scaled[0].size)
# data_set_scaled=data_set.values
backcandles = 15
print(data_set_scaled.shape[0])

for j in range(8): # data_set_scaled[0].size):#2 columns are target not X
    X.append([])
    for i in range(backcandles, data_set_scaled.shape[0]): # backcandles+2
        X[j].append(data_set_scaled[i-backcandles:i, j])

#move axis from 0 to position 2
# Converts the list of arrays into a 3D ndarray array.
X=np.moveaxis(X, [0], [2])
# X is not used.

#Erase first elements of y because of backcandles to match X length
#del(yi[0:backcandles])
#X, yi = np.array(X), np.array(yi)
# Choose -1 for last column, classification else -2...
X, yi =np.array(X), np.array(data_set_scaled[backcandles:,-1])
y=np.reshape(yi,(len(yi),1))
#y=sc.fit_transform(yi)
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

print("data_set.shape:",data_set.shape,"X.shape:",X.shape)
print(X)
print("=========================================================")
print(y.shape)
print(y)


2390
data_set.shape: (2390, 19) X.shape: (2375, 15, 8)
[[[0.00000000e+00 1.63884959e-01 1.64926093e-01 ... 1.71771279e-01
   1.05248237e-02 9.91909682e-01]
  [4.18760469e-04 1.73842934e-01 1.71164282e-01 ... 1.71418421e-01
   9.04975371e-03 9.74889637e-01]
  [8.37520938e-04 1.72019670e-01 1.72566119e-01 ... 1.76358435e-01
   8.59350083e-03 9.82260122e-01]
  ...
  [5.02512563e-03 1.50771388e-01 1.40393955e-01 ... 1.49047249e-01
   0.00000000e+00 3.59521603e-01]
  [5.44388610e-03 1.38218777e-01 1.27847502e-01 ... 1.29216638e-01
   8.41631525e-05 1.25942202e-01]
  [5.86264657e-03 1.35904663e-01 1.26445665e-01 ... 1.34086097e-01
   4.42963960e-06 2.93021867e-01]]

 [[4.18760469e-04 1.73842934e-01 1.71164282e-01 ... 1.71418421e-01
   9.04975371e-03 9.74889637e-01]
  [8.37520938e-04 1.72019670e-01 1.72566119e-01 ... 1.76358435e-01
   8.59350083e-03 9.82260122e-01]
  [1.25628141e-03 1.78891988e-01 1.68781143e-01 ... 1.70642150e-01
   5.87813176e-03 6.71946116e-01]
  ...
  [5.44388610e-03 1.38

In [8]:
#also comprehensions for X
#X = np.array([data_set_scaled[i-backcandles:i,:4].copy() for i in range(backcandles,len(data_set_scaled))])
#print(X)
#print(X.shape)

In [9]:
# split data into train test sets
splitlimit = int(len(X)*0.8)
print("lenX:",len(X), "splitLimit:",splitlimit)
X_train, X_test = X[:splitlimit], X[splitlimit:] # Training data, Test Data
y_train, y_test = y[:splitlimit], y[splitlimit:] # Training data, Test Data
adj_close_train, adj_close_test = data_set.loc[:splitlimit, 'Adj Close':'Adj Close'], data_set.loc[splitlimit:, 'Adj Close':'Adj Close'] # Training data, Test Data
print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)
print("X_test.shape:", X_test.shape)
print("y_test.shape:", y_test.shape)
print("adj_close_test.shape:", adj_close_test.shape)
print("=============================================")
print("y_train:", y_train)

lenX: 2375 splitLimit: 1900
X_train.shape: (1900, 15, 8)
y_train.shape: (1900, 1)
X_test.shape: (475, 15, 8)
y_test.shape: (475, 1)
adj_close_test.shape: (490, 1)
y_train: [[0.11870145]
 [0.11947772]
 [0.12399432]
 ...
 [0.48362733]
 [0.48398019]
 [0.47713477]]


# Load an Existing Model -or- Model Training
 - Using LSTM (Long Short Term Memory) Model
 - Using Keras (Tensorflow) Library
 - LSTM --> Dense Layer --> Activation Layer


In [10]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed

import tensorflow as tf
import keras
from keras import optimizers
from keras.callbacks import History
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
import numpy as np
import os

# If the Keras model file exists, load it. Otherwise, create a new model.
if os.path.exists(model_path):
    # Load the model...
    model = keras.models.load_model(model_path)
    print("Model Loaded:", model_path)
else:
    # Create a new model...
    
    #tf.random.set_seed(20)
    np.random.seed(10)
    
    lstm_input = Input(shape=(backcandles, 8), name='lstm_input')
    inputs = LSTM(200, name='first_layer')(lstm_input)
    inputs = Dense(1, name='dense_layer')(inputs)
    output = Activation('linear', name='output')(inputs)
    model = Model(inputs=lstm_input, outputs=output)
    adam = optimizers.Adam()
    model.compile(optimizer=adam, loss='mse')
    # model.fit(x=X_train, y=y_train, batch_size=15, epochs=30, shuffle=True, validation_split = 0.1)
    fitted_model = model.fit(x=X_train, y=y_train, batch_size=15, epochs=50, shuffle=True, validation_split = 0.1)
    print("adj_close_test.shape:", adj_close_test.shape)

    # Save the model...
    model.save(model_path) 

pass

2024-07-26 22:12:18.254291: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-26 22:12:18.264533: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 22:12:18.276519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 22:12:18.280111: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-26 22:12:18.289719: I tensorflow/core/platform/cpu_feature_guar

Model Loaded: models/MGC=F_2015-01-01_2024-07-26.keras


I0000 00:00:1722057139.439060  369178 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 22:12:19.472175: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


# Model Testing

In [11]:
y_pred = model.predict(X_test)
# y_pred_rescaled = sc.inverse_transform(y_pred)
#y_pred=np.where(y_pred > 0.43, 1,0)
for i in range(10):
    print(y_pred[i], y_test[i])

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
[0.4872137] [0.47007761]
[0.47983286] [0.45786869]
[0.4724929] [0.4673253]
[0.46283853] [0.46055044]
[0.47146925] [0.47099499]
[0.4699648] [0.46563156]
[0.4756572] [0.47141849]
[0.47300544] [0.47988709]
[0.47589594] [0.46316156]
[0.480404] [0.45688067]


## - Adjust the prediction
  The ideal for the prediction is for it to be as close to the actual price as possible.
  What was observed is that the prediction is often off by a delta.
  The delta is the difference between the actual price and the predicted price.
  After applying an average of the delta to the prediction, the prediction is closer to the actual price.
  Usually within 1% of the actual price, which is an incredible improvement.  
  
  The resulting numpy array y_p_adj is the adjusted prediction, which we will use as the prediction for the next day.

In [12]:
# Calculate the delta between actual price and prediction
# Bring the prediction closer to the price based on the delta
def mov_avg(x, w):
    return abs(np.convolve(x, np.ones(w), 'valid') / w)

def apply_deltas(a,p,d, w):
    al = list(a) # Actual
    pl = list(p) # Predicted
    dl = list(d) # Delta
    rl = [] # Return List
    if len(dl) < len(pl):
        for i in range(len(pl) - len(dl)):
            # Duplidate the first value in the list to make the list the same size as pl.
            dl.insert(0, dl[0])
    # We don't apply the average to our first predictions
    # fot the size of the moving average window.
    # For this area, we just return the predicted value.
    i = w - 1
    for j in range(0, i):
        rl.append(pl[j]) 
    print("al.len", len(al), "pl.len", len(pl), "dl.len", len(dl))
    # Beyond the window, we apply the average delta to the prediction.
    for delta in dl:
        if i >= len(dl):
            break
        print("i:", i, "dl:", len(dl), "al:", len(al), "pl:", len(pl))
        if al[i] > pl[i]:
            # If the actual price is greater than the predicted price, add the delta.
            val = pl[i] + delta
        else:
            # If the actual price is less than or equal to the predicted price, subtract the delta.
            val = pl[i] - delta
        rl.append(val)
        i += 1
    return rl

avg_win = 6

y_p_delta = y_pred - y_test
# Shift delta to the left by 1 day so that the adjusted prediction
# only uses know data, and duplidate the delta for the last day (which we don't know).
y_p_delta = np.append(y_p_delta[1:], y_p_delta[-1])
# Calculate the average delta for the prediction.
y_p_davg = mov_avg(y_p_delta, avg_win)

# print(y_test.shape, y_pred.shape, y_p_adj.shape, y_p_delta.shape)
y_p_adj = apply_deltas(y_test, y_pred, y_p_davg, avg_win)
# We don't know the actual price for today, so we don't know the delta and average,
# so for today, we will use the last known average delta (yesterday's) to adjust todays prediction.
if y_test[-2] > y_pred[-1]:
    y_p_adj[-1] = y_pred[-1] + y_p_davg[-2]
else:
    y_p_adj[-1] = y_pred[-1] - y_p_davg[-2]
# Convert y_p_adj to a numpy array for plotting.
y_p_adj = np.array(y_p_adj)

print("adj_close_test.shape:", adj_close_test.shape)
print("y_pred:", y_pred.shape[0])


al.len 475 pl.len 475 dl.len 475
i: 5 dl: 475 al: 475 pl: 475
i: 6 dl: 475 al: 475 pl: 475
i: 7 dl: 475 al: 475 pl: 475
i: 8 dl: 475 al: 475 pl: 475
i: 9 dl: 475 al: 475 pl: 475
i: 10 dl: 475 al: 475 pl: 475
i: 11 dl: 475 al: 475 pl: 475
i: 12 dl: 475 al: 475 pl: 475
i: 13 dl: 475 al: 475 pl: 475
i: 14 dl: 475 al: 475 pl: 475
i: 15 dl: 475 al: 475 pl: 475
i: 16 dl: 475 al: 475 pl: 475
i: 17 dl: 475 al: 475 pl: 475
i: 18 dl: 475 al: 475 pl: 475
i: 19 dl: 475 al: 475 pl: 475
i: 20 dl: 475 al: 475 pl: 475
i: 21 dl: 475 al: 475 pl: 475
i: 22 dl: 475 al: 475 pl: 475
i: 23 dl: 475 al: 475 pl: 475
i: 24 dl: 475 al: 475 pl: 475
i: 25 dl: 475 al: 475 pl: 475
i: 26 dl: 475 al: 475 pl: 475
i: 27 dl: 475 al: 475 pl: 475
i: 28 dl: 475 al: 475 pl: 475
i: 29 dl: 475 al: 475 pl: 475
i: 30 dl: 475 al: 475 pl: 475
i: 31 dl: 475 al: 475 pl: 475
i: 32 dl: 475 al: 475 pl: 475
i: 33 dl: 475 al: 475 pl: 475
i: 34 dl: 475 al: 475 pl: 475
i: 35 dl: 475 al: 475 pl: 475
i: 36 dl: 475 al: 475 pl: 475
i: 37 dl: 47

## - Plot the Test Results

### - Unscaled Data (0..1) (Prediction vs Target)

In [13]:
# Use an interactive plot to see the results
%matplotlib notebook
# Plot the scaled test and predicted data.
y_test_plot = y_test[:-1]
plt.figure(figsize=(16,8))
plt.plot(y_test_plot, color = 'black', label = 'Test', marker='.')
# plt.plot(y_pred, color = 'red', label = 'Pred', marker='1')
plt.plot(y_p_adj, color = 'green', label = 'Adj Pred', marker='1')
plt.legend()
plt.grid()
plt.show()


<IPython.core.display.Javascript object>

### - Rescaled Data ($USD) (Prediction vs Target)

In [17]:
# Plot Candlesticks...
def plot_candlesticks(plt, prices, title, width=1, width2=.5, clr_up='green', clr_dn='red'):

    #define up and down prices
    up = prices[prices.Close>=prices.Open]
    down = prices[prices.Close<prices.Open]
    
    #plot up prices
    plt.bar(up.index,up.Close-up.Open,width,bottom=up.Open,color=clr_up)
    plt.bar(up.index,up.High-up.Close,width2,bottom=up.Close,color=clr_up)
    plt.bar(up.index,up.Low-up.Open,width2,bottom=up.Open,color=clr_up)
    
    #plot down prices
    plt.bar(down.index,down.Close-down.Open,width,bottom=down.Open,color=clr_dn)
    plt.bar(down.index,down.High-down.Open,width2,bottom=down.Open,color=clr_dn)
    plt.bar(down.index,down.Low-down.Close,width2,bottom=down.Close,color=clr_dn)
    
    #rotate x-axis tick labels
    plt.xticks(rotation=45, ha='right')


In [86]:
# Plot Candlesticks...
def plot_candlesticks(plt, prices, title, width=1, width2=.25, clr_up='green', clr_dn='red', clr_wick='black'):

    #define up and down prices
    up = prices[prices.Close>=prices.Open]
    down = prices[prices.Close<prices.Open]
    
    #plot up prices
    plt.bar(up.index,up.Close-up.Open,width,bottom=up.Open,color=clr_up, alpha=.25)
    # plt.bar(up.index,up.High-up.Close,width2,bottom=up.Close,color=clr_up, alpha=.25)
    # plt.bar(up.index,up.Low-up.Open,width2,bottom=up.Open,color=clr_up, alpha=.25)
    plt.bar(up.index,up.High-up.Low,width2,bottom=up.Open,color=clr_wick, alpha=.25)

    #plot down prices
    plt.bar(down.index,down.Close-down.Open,width,bottom=down.Open,color=clr_dn, alpha=.25)
    # plt.bar(down.index,down.High-down.Open,width2,bottom=down.Open,color=clr_dn, alpha=.25)
    # plt.bar(down.index,down.Low-down.Close,width2,bottom=down.Close,color=clr_dn, alpha=.25)
    plt.bar(down.index,down.High-down.Low,width2,bottom=down.Close,color=clr_wick, alpha=.25)
    
    #rotate x-axis tick labels
    plt.xticks(rotation=45, ha='right')

# Use an interactive plot to see the results
%matplotlib notebook
import math
import mplfinance as mfp

# Plot the original Adj Cost data that is in dollars...
# Remove the last Adj Price, because it just aopy of the last adj
plt_test_usd = np.array(data_set[backcandles + splitlimit:]['Adj Close'])
plt_test_plot =np.array(plt_test_usd[:-1].reshape(-1,1))

# Using Inverse_Transform to Rescale the predicted data to dollars...
# Not good, because the prediction values returned exactly match the Target value
# rather than the predicted price..
# plt_pred_adj = sc.inverse_transform(np.append(y_p_adj, data_set.to_numpy()[backcandles + splitlimit:, 1:], axis=1))[:, -1]

# Windowed Mean Rescaler
# plt_test_usd_scaler = np.array( pd.DataFrame([ plt_test_usd[i] / y_test[i] for i in range(len(plt_test_usd)) ]).rolling(window=1).mean() )
# Direct Rescaler
plt_test_scaler = np.array( pd.DataFrame([ plt_test_usd[i] / y_test[i] for i in range(len(plt_test_usd)) ]) )

# Rescale the scaled test data (Matches to Original Price Data Exactly)...
plt_test_rescaled = np.array([ (y_test[i] * plt_test_scaler[i]) for i in range(len(y_test)) ])

# Prediction Relative/Delta to Actual Price (not as good as direct)...
# plt_pred_adj_r = [ ((y_p_adj[i] * plt_test_scaler[i]) - (y_test[i] * plt_test_scaler[i])) + plt_test_usd[i] for i in range(len(y_p_adj)) ] 
# Prediction: Direct from Scaled output (better than relative)...
# *** Rescaled the prediction data to dollars look quite different from the unscaled graph...
# plt_pred_adj = np.array([ (y_p_adj[i] * plt_test_scaler[i]) for i in range(len(y_p_adj)) ])
# Prediction values, when rescaled to dollars, can be far off from the actual prices.

# The rescale_factor is used to bring the prediction closer to the actual price.
# Current set manually by eyeing re-scaled charts against an unscaled chart.
# How can this be more accurately calculated?
# - find rescaled prediction deltas that very wildly from the same deltas in the unscaled chart.
# - Calculate scaling for each delta that brings the rescaled prediction deltas closer to the unscaled deltas.
# - Get the mean of the scaling factors. And use that as the rescaling_factor to adjust the prediction.

# # Calculate the rescale factor...
# # - Get unscaled deltas between the prediction and the actual price.
# # - Get rescaled deltas between the prediction and the actual price.
# # - Get top 20% of all scaled-deltas swing wildly from the unscaled-deltas.
# *** Not really helpful, as the rescale factor calculated results in the same bad rescaled prediction values.
# scaled_delta_pct = np.array([ (y_p_adj[i] - y_test[i]) / y_test[i] for i in range(len(y_p_adj)) ])
# rescaled_delta_pct = np.array([ (((y_p_adj[i] * plt_test_scaler[i])  - (y_test[i] * plt_test_scaler[i])) / (y_test[i] * plt_test_scaler[i])) for i in range(len(y_p_adj)) ])
# rescale_factor = np.array([ (rescaled_delta_pct[i] / (scaled_delta_pct[i] / 5)) for i in range(len(y_p_adj)) ])
# plt_pred_adj = np.array([ (((y_p_adj[i] - y_test[i]) * plt_test_scaler[i]/rescale_factor[i]) + plt_test_rescaled[i])  for i in range(len(y_p_adj)) ])

# An arbitrary rescale factor works best...
rescale_factor = math.pi
plt_pred_adj = np.array([ (((y_p_adj[i] - y_test[i]) * plt_test_scaler[i]/rescale_factor) + plt_test_rescaled[i])  for i in range(len(y_p_adj)) ])

# # # Calculate the mean of the delta between the prediction and the actual price...
# # # Used to create price range circles around the prediction...
plt_delta = np.array(pd.DataFrame([abs(plt_test_usd[i] - plt_pred_adj[i]) for i in range(len(y_p_adj)) ]).rolling(window=6).mean())
# # # Drop all but the first column...
# # plt_delta = np.delete(plt_delta, np.s_[1:], axis=1)
# # plt_delta = np.array(pd.DataFrame(abs(plt_test_usd - plt_pred_adj)).rolling(window=0).mean())
# 
# # Adusted prediction prices are too far off from the actual prices after rescaling to dollars.
# # So bring the prediction closer to the actual price based on the delta...
# plt_prod_adj = apply_deltas(plt_test_usd, plt_pred_adj, plt_delta, 0)

# Dates...
data_date_str = [str(date) for date in data_date]
plt_test_dates = np.array((data_date_str[backcandles + splitlimit - 1:-1]), ).reshape(-1,1)
plt_pred_dates = np.array((data_date_str[backcandles + splitlimit - 1:]),   ).reshape(-1,1)

# empty_arr = np.zeros(((len(plt_test_plot)+1, 18)))
# plt_pred_adj = sc.inverse_transform(np.append(y_p_adj, empty_arr, axis=1))[0:, 1]

# Plot the data...
plt.figure(figsize=(16,8))
plt.autoscale(tight=True)

plt_test_plot = plt_test_plot.reshape(-1,1)

# Plot the actual price data ($USD)...
plt.plot(plt_test_plot, color = 'black', label = 'Test', marker='+')
# Plot the adj predicted price data ($USD)...
plt.plot(plt_pred_adj, color = 'green', label = 'Test', marker='.')

# Plot the candlesticks...
ds_tmp = data_set[backcandles + splitlimit:-1]
cs_data = ds_tmp.loc[backcandles + splitlimit:, ['Open','High','Low','Close']].copy().reset_index()
plot_candlesticks (plt, cs_data, 'Candlesticks')

# plt.xlabel(range(0,len(plt_pred_dates)), plt_pred_dates.tolist(), rotation=45)
#  .set_major_formatter(matplotlib.dates.DateFormatter('%Y')))

# Plot a Price Range Circle Arounds the current Prediction...
Plot_Pred_Circle = True
if Plot_Pred_Circle:
    import matplotlib.patches as ptch
    lst_pred = len(plt_pred_adj) -1
    # ax.set_xlim = ([0, len(plt_pred_adj)])
    # ax.set_ylim = ([plt_pred_adj.min(), plt_pred_adj.max()])
    cpos = np.array([lst_pred, plt_pred_adj[-1][0]])
    circ_pred0 = ptch.Circle(cpos, radius=plt_delta[-1][0]/1,   color='grey',   fill=True, alpha=0.20) 
    circ_pred1 = ptch.Circle(cpos, radius=plt_delta[-1][0]/1.5, color='blue',   fill=True, alpha=0.20) 
    circ_pred2 = ptch.Circle(cpos, radius=plt_delta[-1][0]/2,   color='yellow', fill=True, alpha=0.20) 
    circ_pred3 = ptch.Circle(cpos, radius=plt_delta[-1][0]/3,   color='red',    fill=True, alpha=0.20) 
    ax = plt.gca()
    # ax.set_aspect("equal")
    ax.add_patch(circ_pred0)
    ax.add_patch(circ_pred1)
    ax.add_patch(circ_pred2)
    ax.add_patch(circ_pred3)

plt.grid(visible=True, which='both')
plt.legend()
plt.show()

pass

<IPython.core.display.Javascript object>

# Analysis of the Prediction
 - Rescaled data (0..1) (Prediction vs Target)

In [ ]:
# We convert our data back to dollars...
# predval = y_pred[i] * scalefac

# Convert back to dollar $values
elements = len(y_pred)
print(elements)
tot_deltas = 0
tot_tradrng = 0
for i in range(-1, -elements, -1):
    actual = data_set['Adj Close'].iloc[i - 1]
    scalefac = data_set['Adj Close'].iloc[i - 1] / y_test[i - 1]
    # print("Scaling Factor", scalefac)
    predval = y_pred[i] * scalefac
    predval = y_pred[i - 1] if np.isinf(predval) else predval
    pred_delta = abs(predval - actual)
    tot_deltas += pred_delta
    trd_rng = abs(data_set['High'].iloc[i - 1] - data_set['Low'].iloc[i])
    tot_tradrng += trd_rng
    print("Predicted: ", predval, "  Actual:", actual, "  Delta:", pred_delta, "  Trade Rng:", trd_rng)
print("Mean Trading Range:", round(tot_tradrng / elements, 2))
print("Mean Delta:", round(float(tot_deltas[0] / elements), 2))

## - Rescaled data (0..1) (Adjusted Prediction vs Target)

In [ ]:
elements = len(y_p_adj)
print(elements)
tot_deltas = 0
tot_tradrng = 0
for i in range(-1, -elements, -1):
    actual = data_set['Adj Close'].iloc[i - 1]
    scalefac = data_set['Adj Close'].iloc[i - 1] / y_test[i - 1]
    # print("Scaling Factor", scalefac)
    predval = y_p_adj[i] * scalefac
    predval = y_pred[i - 1] if np.isinf(predval) else predval
    pred_delta = abs(predval - actual)
    tot_deltas += pred_delta
    trd_rng = abs(data_set['High'].iloc[i - 1] - data_set['Low'].iloc[i])
    tot_tradrng += trd_rng
    print("Predicted: ", predval, "  Actual:", actual, "  Delta:", pred_delta, "  Trade Rng:", trd_rng)
print("Mean Trading Range:", round(tot_tradrng / elements, 2))
print("Mean Delta:", round((tot_deltas[0] / elements), 2))

### - Deltas between predicted price and actual price

In [ ]:
# Calculate an array of deltas between predicted price and actual price
print("Std Dev -     Pred: ", pred_delta.std())
y_pa_delta = abs(y_test - y_p_adj)
print("Std Dev - Pred Adj: ", y_pa_delta.std())
print("Better by", (y_p_delta.std() / y_pa_delta.std()), "Standard Deviations")

## - Rescaled data (0..1) (Adj Prediction + Adj Rescale vs Target)

In [ ]:
elements = len(plt_pred_adj)
print(elements)
tot_deltas = 0
tot_tradrng = 0
for i in range(-1, -elements, -1):
    actual = data_set['Adj Close'].iloc[i - 1]
    predval = plt_pred_adj[i]
    pred_delta = abs(predval - actual)
    tot_deltas += pred_delta
    trd_rng = abs(data_set['High'].iloc[i - 1] - data_set['Low'].iloc[i])
    tot_tradrng += trd_rng
    print("Predicted: ", predval, "  Actual:", actual, "  Delta:", pred_delta, "  Trade Rng:", trd_rng)
print("Mean Trading Range:", round(tot_tradrng / elements, 2))
print("Mean Delta:", round((tot_deltas[0] / elements), 2))

### - Deltas between predicted price and actual price

In [ ]:
# Calculate an array of deltas between predicted price and actual price
print("Std Dev -     Pred: ", pred_delta.std())
y_pa_delta = abs(y_test - y_p_adj)
print("Std Dev - Pred Adj: ", y_pa_delta.std())
print("Better by", (y_p_delta.std() / y_pa_delta.std()), "Standard Deviations")

## - Trend Prediction Stats using Unscaled (v < 1) Adjusted Prediction

In [ ]:
# How often is the prediction within 1% of the actual price?
pct = 0.05
pct_delta = abs(y_test * pct)
delta = abs(y_test - y_p_adj)
delta = delta[delta < pct_delta]
# Get average price range for the instrument
avg_price_rng = data_set['High'] - data_set['Low']
# Get pct of average price range
pcnt_avg_price_rng = avg_price_rng.mean() * pct

print("Within",pct * 100,"%:","[ Trading Range:$", "Within Avg Price:", avg_price_rng.mean().round(2),pcnt_avg_price_rng.round(2),"]", round((len(delta) / len(y_test)),2))

# How often is the predicted trend the same as the actual trend?
test_trend = y_test[1:] - y_test[:-1]
pred_trend = y_p_adj[1:] - y_p_adj[:-1]
delta = [1 if (test_trend < 0) and (pred_trend < 0) or (test_trend > 0) and (pred_trend > 0) else 0 for test_trend, pred_trend in zip(test_trend, pred_trend)]
# Print the number of elements in delta and the percent of delta that are 1.
print("Same Trend:", len(delta), "Wrong:", len(delta) - np.count_nonzero(delta), "Right", np.count_nonzero(delta), "%Right", round(sum(delta) / len(delta) * 100,2))


## - Trend Prediction Stats using Rescaled ($USD) Adjusted Prediction

In [ ]:
# How often is the prediction within 1% of the actual price?
pct = 0.05
pct_delta = abs(plt_test_plot * pct)
delta = abs(plt_test_plot - plt_pred_adj[:-1])
delta = delta[delta < pct_delta]
# Get average price range for the instrument
avg_price_rng = data_set['High'] - data_set['Low']
# Get pct of average price range
pcnt_avg_price_rng = avg_price_rng.mean() * pct

print("Within",pct * 100,"%:","[ Trading Range:$", "Within Avg Price:", avg_price_rng.mean().round(2),pcnt_avg_price_rng.round(2),"]", round((len(delta) / len(y_test)),2))

# How often is the predicted trend the same as the actual trend?
test_trend = plt_test_plot[1:] - plt_test_plot[:-1]
pred_trend = plt_pred_adj[1:] - plt_pred_adj[:-1]
delta = [1 if (test_trend < 0) and (pred_trend < 0) or (test_trend > 0) and (pred_trend > 0) else 0 for test_trend, pred_trend in zip(test_trend, pred_trend)]
# Print the number of elements in delta and the percent of delta that are 1.
print("Same Trend:", len(delta), "Wrong:", len(delta) - np.count_nonzero(delta), "Right", np.count_nonzero(delta), "%Right", round(sum(delta) / len(delta) * 100,2))


# Model Details...

In [ ]:
# Review feature importance
# importance = model.coef_

import shap
import pydot

print(X_train.shape)

print(model.summary())

from keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
from IPython.display import Image

pass

## - Model Plot
<img src="model_plot.png" alt="Drawing" style="width: 200px;"/>